# Executando um Processo de WebScraping para Extrair Informações da Rede Social LinkedIn

O Web Scraping nada mais é que uma extração de dados na internet de forma automatizada, amplamente usada ao redor do mundo para capturar todo tipo de informação relevante para tomadas de decisões, direcionamento de marketing, estudo de concorrência pessoal ou empresarial, desenvolver sistemas de recomendação de produtos ou serviços, e para diversas outras finalidades. 

As possibilidades e o poder desse processo são quase ilimitados, onde basicamente tudo que é visível na Web é possível ser extraído, e portanto o famoso conselho do Tio Ben no filme do Homem-Aranha (2002) se aplica tão bem ao Web Scraping:

<center><img alt="Famosa frase do Tio Ben" src='https://pm1.narvii.com/6409/5097b0d352a5cb408028a0208de49116efaedd37_hq.jpg'></center>

Fonte: Amino.

Todos os dias deixamos rastros e informações pessoais visíveis na internet, seja pelos mecanismos de buscas ou principalmente pelas redes sociais, que para muitas pessoas é um local de compartilhar seus momentos de vida, ou melhor, compartilhar seus dados para todas as pessoas na rede, que para aqueles que possuem intenções maliciosas é um prato cheio ter acesso a essas informações. Por isso, a frase do Tio Ben é tão importante para o Web Scraping, da mesma forma que podem ser desenvolvidas aplicações incríveis, também podem ser desenvolvidas aplicações maliciosas (por exemplo: venda de informações pessoais).

Como a maioria das grandes organizações reconhecem o valor dessas informações de seus usuários, políticas contra o uso do Web Scraping foram desenvolvidas para tentar controlar o que pode, ou não, ser extraído de seus produtos, aplicando penalidades para aqueles que quebrem essas políticas. Além disso, muitas delas vêm dificultando o processo de Scraping nos últimos anos com certa frequência, como é o caso do LinkedIn, que será a rede social que iremos abordar nesse projeto.

Para aqueles que tentaram realizar o processo de Web Scraping no LinkedIn, provavelmente ficaram conhecendo algumas das lutas dessa rede profissional para proibir o uso de Scraping para extrair dados de seus usuários, sendo o caso mais conhecido com a hiQ Labs, que possui uma história de quase 6 anos na justiça.

A ação judicial teve início em 2017, quando o LinkedIn acusou a hiQ Lab de coletar dados de usuários da rede social de forma abusiva, onde a hiQ Labs estaria usando as informações dos usuários para desenvolver algoritmos que poderiam prever quando os funcionários poderiam deixar seus empregos, porém o LinkedIn sofreu uma derrota quando a concorrente processou a rede afirmando que os dados públicos são de fato públicos e que a rede LinkedIn não poderia impedir que qualquer empresa possa ter acesso a esses dados. Entretanto, recentemente, outra ação foi levantada pelo LinkedIn que afirma que os Bots da hiQ Labs estariam filtrando e coletandos dados que nenhum humano conseguiria coletar, ou seja, dados não públicos, que ainda estariam sendo colocados à venda por trás dos panos.

Não entrarei mais a fundo sobre essa luta judicial por não ser o foco deste projeto, para isso separei alguns artigos sobre o assunto na Seção Referências ao fim do Projeto, mas já adianto uma coisa para despertar seu interesse em lê-los: o LinkedIn ganhou a disputa judicial recentemente!

O problema que iremos simular neste projeto para que o Web Scraping seja a solução é:

"***O Setor de Recursos Humanos da empresa divulga as vagas de emprego disponíveis no LinkedIn e consegue ter acesso às páginas dos membros que aplicaram para essas vagas, porém acessar cada página uma por uma para encontrar o melhor candidato é algo trabalhoso e que demanda bastante tempo, ainda mais quando a vaga recebe mais de 200 aplicações (o que é bastante comum vermos até maiores), posto essa dificuldade, será que é possível obter as informações de todas essas pessoas que aplicaram para as vagas de maneira mais prática, como, por exemplo, em uma única Tabela?***"

O objetivo deste projeto é desenvolver um processo de Web Scraping no LinkedIn para extrair algumas informações da página de um membro da rede e salvar as informações em um Pandas DataFrame que solucionará essa dor da equipe de RH do negócio, e para isso utilizaremos apenas duas bibliotecas: **Selenium** e **BeautifulSoup**.

Como comentei anteriormente, o LinkedIn vem tomando medidas protetivas contra extrações de dados na Plataforma com uma certa frequência, portanto o código utilizado aqui pode não ser funcional em questão de dias, além disso tenha em mente que precisamos de uma conta ativa na plataforma para que possamos extrair os dados dos membros, na qual pode ter Features que venham a ser bloqueadas ou até mesmo suspensão total da conta, então tome cuidado para não abusar nas extrações.

## Preparação do Processo

Primeiramente é preciso fazer o download do [WebDriver](https://chromedriver.chromium.org/downloads), que é responsável por realizar o processo automatizado do Web Scraping, e após a instalação importamos as bibliotecas:

In [1]:
# Importando as bibliotecas
!pip install selenium -q
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

!pip install beautifulsoup4 -q
from bs4 import BeautifulSoup

!pip install pandas -q
import pandas as pd

import time
from datetime import date

## Instanciando o WebDriver

Para criar a instância do WebDriver é necessário passar o caminho no qual ele foi instalado, no meu caso na própria pasta do projeto.

In [2]:
# Criando a instância no WebDriver
srv = Service('CAMINHO DO WEB DRIVER\chromedriver.exe') 
opt = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=srv, options=opt)

## Logando em sua Conta no LinkedIn

Aqui iremos acessar a página de Login da Plataforma e passar os dados da conta.

In [3]:
# Abrindo a página de Login do LinkedIn
driver.get('https://www.linkedin.com/login/')
time.sleep(1) # aguardar 1 segundos para a página abrir

# Inserindo o usuário
usuario = driver.find_element('id','username') # acessando o campo usuário
usuario.send_keys('SEU LOGIN') # enviando a string com o usuário

# Inserindo a senha
senha = driver.find_element('id','password') # acessando o campo senha
senha.send_keys('SUA SENHA') # enviando a string com a senha

# Clicando no botão 'Entrar' para acessar a conta no LinkedIn
driver.find_element("xpath","//button[@type='submit']").click()

## Acessando um Perfil através da URL do Usuário

Como descrito em nossa situação problema, temos as URLs dos candidatos que aplicaram às vagas postadas no LinkedIn pela equipe de RH do negócio, mas por enquanto vamos testar nosso processo de Web Scraping para apenas um único candidato.

In [4]:
# Acessando o perfil
perfil_url = 'https://www.linkedin.com/in/crisjunqueira/' # ou 'https://www.linkedin.com/in/williamhgates/'
driver.get(perfil_url)

## Explorando uma Feature do LinkedIn (Baixar Perfil em PDF)

Uma Feature interessante do LinkedIn é a possibilidade de baixar as informações do perfil em um formato de currículo em PDF, então como um extra adicionaremos essa funcionalidade ao nosso Scraping, ou seja, para cada perfil teremos um 'Currículo' em PDF que pode ser explorado com outras ferramentas de extração de dados em documentos.

Alguns perfis possuem diferentes layouts de página, portanto veremos várias partes do código com `TRY`/`EXCEPT` para buscar generalizar o código para mais formatos de página.

In [6]:
try:
    # Clicando no botão '...' ou 'Mais' na seção de "Introdução" do perfil
    driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[3]/button').click()
    time.sleep(1) # aguardar 1 segundo

    # Clicando no botão "Salvar como PDF"
    driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[3]/div/div/ul/li[3]/div').click()

except:
    # Clicando no botão '...' ou 'Mais' na seção de "Introdução" do perfil
    driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button').click()
    time.sleep(1) # aguardar 1 segundo

    # Clicando no botão "Salvar como PDF"
    driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[3]/div').click()

## Quais Informações devemos extrair?

Voltando a nossa situação problema, digamos que a equipe de RH nos passou uma lista de quais informações são chaves para a tomada de decisão sobre o candidato, que são:

* Nome Candidato;
* Título do Candidato;
* Organização Atual;
* Localização do Candidato;
* Biografia;
* Título da Última Experiência Profissional;
* Nome da Organização da Última Experiência Profissional;
* Tipo de Contratação da Organização;
* Data de Início da Experiência;
* Data de Término da Experiência;
* Duração da Experiência;
* Instituição referente Última Formação Acadêmica;
* Tipo da Formação Acadêmica;
* Título da Formação Acadêmica;
* Duração da Formação Acadêmica;
* Número de Seguidores;
* Idiomas;
* Prêmios e Honrarias;
* Quantidade de Recomendações.

Como um extra para gestão de base de dados, vamos acrescentar as seguintes informações:
* Data de Ingestão (quando os perfis foram inseridos na base);
* Disponibilidade da URL para consultas manuais (caso o recrutador queira acessar o perfil do candidato);

## Obtendo o Código Fonte HTML da Página

Antes de extrairmos as informações, precisamos obter o código fonte da página em HTML e para isso iremos fazer com que o Driver desça até o fim da página e depois criaremos um Objeto com a biblioteca **BeautifulSoup**, no qual usaremos para poder acessar as informações no código fonte:

In [5]:
# Lendo a página do início ao fim
inicio = time.time()
posicao_inicial_rolamento = 0
posicao_final_rolamento = 1000

while True:
    driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")
   
    posicao_inicial_rolamento = posicao_final_rolamento
    posicao_final_rolamento += 1000
    
    # Aguardando 3 segundos
    time.sleep(3)
    
    fim = time.time()
    
    # Executar o script por 20 segundos
    if round(fim - inicio) > 20:
        break

In [6]:
# Salvando o código fonte da página em uma variável
src = driver.page_source

# Utilizando o código fonte para gerar um objeto Beautiful Soup
soup = BeautifulSoup(src,'lxml')

## Extraindo as Informações da Introdução do Perfil

<center><img alt="Exemplo de Introdução no LinkedIn" src='https://github.com/octavianosilva/Imagens_projetos/blob/main/Introdu%C3%A7%C3%A3o%20do%20Perfil.png?raw=true'></center>

Para acessarmos as informações dentro do Objeto com o código fonte, usamos o `find()`, passando o elemento e a classe de onde está localizada a seção Introdução (para localizar esses parâmetros basta inspecionar os elementos na página do driver).

Uma das complicações do processo de extração está na identificação de onde estão as informações que queremos, pois muitos elementos no código fonte da página mudam conforme a sua seção de login, dessa forma localizar os elementos corretos que não sofrem alterações é um trabalho árduo.

In [7]:
# Obtendo o HTML de extração da introdução
intro = soup.find('div',{'class': 'mt2 relative'})

# Visualizando o código HTML
#print(intro)

Agora que temos o código localizado da Seção Introdução, podemos extrair as informações: Nome do Candidato, Título Profissional, Localização, Organização Atual e Formação Acadêmica. Como podem haver perfis de candidatos desempregados e também que não possuem formação acadêmica, esses campos precisarão ser condicionados.

In [10]:
# Localizando e extraindo o nome do candidato
nome_loc = intro.find('h1')
nome = nome_loc.get_text().strip()

# Localizando e extraindo o título profissional do candidato
titulo_loc = intro.find('div', {'class': 'text-body-medium break-words'})
titulo = titulo_loc.get_text().strip()

# Localizando e extraindo o endereço de residência
localizacao_loc = intro.find('span', {'class': 'text-body-small inline t-black--light break-words'})
localizacao = localizacao_loc.get_text().strip()

# Localizando e extraindo a organização de trabalho
trabalho_loc = intro.find('div', {'class': 'inline-show-more-text'})
if trabalho_loc == None:
    trabalho = ''
else:
    trabalho = trabalho_loc.get_text().strip()

# Localizando e extraindo a instituição da formação acadêmica
estudo_loc = intro.find_all('div', {'class': 'inline-show-more-text'})
if estudo_loc == None:
    estudo = ''
else:
    estudo = estudo_loc[1].get_text().strip()

# Exibindo os resultados obtidos até agora
print('Nome:',nome,
      '\nTítulo:', titulo,
      '\nLocalização:', localizacao,
      '\nTrabalha em:', trabalho,
      '\nEstudou em:', estudo)

Nome: Cristina Junqueira 
Título: Co-Founder at Nubank 
Localização: São Paulo, São Paulo, Brasil 
Trabalha em: Nubank 
Estudou em: Northwestern University - Kellogg School of Management


## Extraindo a Quantidade de Seguidores da Página

Pode ser interessante conhecer o alcance do candidato, se ele é influencer, ou se tem muitas ou poucas conexões.

In [8]:
# Obtendo o HTML de extração da quantidade de seguidores
networking = soup.find('ul', {'class':'pv-top-card--list pv-top-card--list-bullet display-flex mt2'}).find('li',{'class':'text-body-small t-black--light inline-block'})

# Visualizando o código HTML
#print(networking)

In [12]:
# Localizando e Extraindo a quantidade de seguidores
seguidores_loc = networking.find('span', {'class': 't-bold'})
seguidores = seguidores_loc.get_text().strip()

# Exibindo a quantidade de seguidores
print('O perfil possui:',seguidores)

O perfil possui: 505.607 seguidores


## Extraindo as Informações da Biografia

Essa informação estã em uma Seção chamada 'Sobre' no perfil e como também há perfis que não possuem uma Biografia, também teremos que condicionar essa informação. 

Lembra que comentei sobre os diferentes layouts das páginas dos usuários quando fizemos o Download do Perfil em PDF? Bom, o mesmo problema acontece aqui, em alguns casos o código abaixo funciona corretamente, porém em outros acaba pegando informações erradas, ou seja, é necessário tratar a base de dados gerada após o processo de Web Scraping.

In [13]:
# Obtendo o HTML de extração da biografia do candidato
sobre_descricao = soup.find('div',{'class':'inline-show-more-text inline-show-more-text--is-collapsed'})

# Visualizando o código HTML
#print(sobre_descricao)

In [14]:
# Localizando e extraindo a descrição de perfil na seção Sobre da página
if sobre_descricao == None:
    sobre = ''
else:
    sobre_loc = sobre_descricao.find('span', {'aria-hidden': 'true'})
    sobre = sobre_loc.get_text().strip()

# Visualizando o conteúdo da secção Sobre da página
print(sobre)

Selected to One Year Accelerated ProgramMajors - Finance and Marketing


Como podemos ver acima, o perfil em questão infelizmente está pegando uma informação presente na área de formação acadêmica, ou seja, informação no lugar errado, porém com base em outros testes realizados com outros perfis, o código consegue pegar as biografias que realmente estão disponíveis na página, sendo por esse motivo não irei me aprofundar na complexidade de generalizar essa informação para todos os tipos de páginas. 

## Extraindo as Informações referentes à última Experiência Profissional

Essa é uma das seções que começam a complicar bastante a extração do Web Scraping, já que também temos perfis com diferentes layouts nessa seção, por exemplo:

<center><img alt="Exemplo de Experiência no LinkedIn" src='https://github.com/octavianosilva/Imagens_projetos/blob/main/exemplo%20de%20experiencia.png?raw=true'></center>

Como é possível observar na imagem acima, há três formas de uma pessoa inserir as informações sobre suas experiências, as vezes com informações básicas, outras com descrições das atividades realizadas e também casos como o primeiro item, onde dentro de uma mesma organização a pessoa passou por diversos cargos. Todas essas mudanças de estrutura tornam difícil a generalização do processo de Web Scraping, caindo sempre no mesmo problema que é nem sempre conseguir pegar as informações corretas.

A seção Experiência possui uma página exclusiva, o que torna mais fácil extrair as informações do que se fossem extraí-las da página principal, então vamos acessar essa página referente a Experiência Profissional.

Como estamos abrindo uma nova página, é importante executar o rolamento dela até o final para pegar todas as informações:

In [9]:
# Acessando a página de Experiência Profissional
driver.get('https://www.linkedin.com/in/crisjunqueira/details/experience/')

# Lendo a página do início ao fim
inicio = time.time()
posicao_inicial_rolamento = 0
posicao_final_rolamento = 1000

while True:
    driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")
   
    posicao_inicial_rolamento = posicao_final_rolamento
    posicao_final_rolamento += 1000
    
    # Aguardando 3 segundos
    time.sleep(3)
    
    fim = time.time()
    
    # Executar o script por 20 segundos
    if round(fim - inicio) > 20:
        break

# Salvando o código fonte da página em uma variável
src_experiencia = driver.page_source

# Utilizando o código fonte para gerar um objeto Beautiful Soup
soup_experiencia = BeautifulSoup(src_experiencia,'lxml')

In [10]:
# Obtendo o HTML de extração das experiências
experiencia = soup_experiencia.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})

# Visualizando o código HTML
#print(experiencia[0])

# Caso queira outras experiências mais antigas, basta mudar a posição na variável 'experiência[posição]'

In [90]:
# Função para uma determinada estrutura de página
def extraindo_info_experiencias(experiencia):
    # Obtendo o título da experiência mais recente
    titulo_experiencia_loc = experiencia[0].find('span',{'class':'mr1 t-bold'}).find('span',{'aria-hidden':'true'})
    titulo_experiencia = titulo_experiencia_loc.get_text().strip()
    
    # Separando o nome da organização e o tipo de contrato
    organizacao_tipo_experiencia_loc = experiencia[0].find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
    organizacao_tipo_experiencia = organizacao_tipo_experiencia_loc.get_text().strip()
    organizacao_tipo_experiencia = organizacao_tipo_experiencia.split('·')
    
    # Nome da organização mais recente
    organizacao_experiencia = organizacao_tipo_experiencia[0]

    # As vezes também temos perfis que não possuem o tipo da contratação especificada no perfil, portanto precisamos de mais um TRY/EXCEPT para contornar esse problema
    try:
        tipo_contrato_experiencia = organizacao_ultima_experiencia[1].strip()
    except:
        tipo_contrato_experiencia = ''
    
    # Separando as datas de início e fim da duração da experiência
    tempo_duracao_experiencia_loc = experiencia[0].find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
    tempo_duracao_experiencia = tempo_duracao_experiencia_loc.get_text().strip().split('·')
    tempo_experiencia = tempo_duracao_experiencia[0].split('-')
    
    # Data de início e fim da experiência
    data_inicio_experiencia = tempo_experiencia[0]
    data_fim_experiencia = tempo_experiencia[1]
    
    # Duração da experiência
    duracao_experiencia = tempo_duracao_experiencia[1]
    
    # Retornando as variáveis com as informações extraídas
    return titulo_experiencia, organizacao_experiencia, tipo_contrato_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia

In [18]:
# Função para conseguir extrair as informações de outras estruturas de páginas
def extraindo_info_experiencias_2(experiencia):
    # Obtendo o nome da organização da experiência mais recente
    organizacao_experiencia_loc = experiencia[0].find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    organizacao_experiencia = organizacao_experiencia_loc.get_text().strip()
    
    # Obtendo o tipo de contrato da experiência (nos casos de pessoas que não tenham essa informação, podem aparecer informações erradas)
    tipo_experiencia_loc = experiencia[0].find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
    tipo_experiencia = tipo_experiencia_loc.get_text().strip().split('·')
    tipo_experiencia = tipo_experiencia[0].strip()
    
    # Obtendo o título mais recente da experiência
    titulo_experiencia_loc = experiencia[0].find_all('a')
    titulo_experiencia_loc = titulo_experiencia_loc[2].find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    titulo_experiencia = titulo_experiencia_loc.get_text().strip()
    
    # Separando as datas de início e fim da duração da experiência
    tempo_duracao_experiencia_loc = experiencia[0].find_all('a')
    tempo_duracao_experiencia_loc = tempo_duracao_experiencia_loc[2].find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
    tempo_duracao_experiencia_loc = tempo_duracao_experiencia_loc.get_text().strip().split('·')
    tempo_experiencia = tempo_duracao_experiencia_loc[0].split('-')
    
    # Datas de início e fim da experiência
    data_inicio_experiencia = tempo_experiencia[0]
    data_fim_experiencia = tempo_experiencia[1]
    
    # Duração da experiência
    duracao_experiencia = tempo_duracao_experiencia_loc[1]
    
    # Retorne as variáveis com as informações extraídas
    return titulo_experiencia, organizacao_experiencia, tipo_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia

In [91]:
# Executando as funções
try:
    try:
        titulo_experiencia, organizacao_experiencia, tipo_contrato_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia = extraindo_info_experiencias(experiencia)
    except:
        titulo_experiencia, organizacao_experiencia, tipo_contrato_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia = extraindo_info_experiencias_2(experiencia)
except:
    titulo_experiencia, organizacao_experiencia, tipo_contrato_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia = '','','','','',''

In [92]:
# Exibindo o resultado da extração
print('Título registrado na última experiência profissional:',titulo_experiencia,
      '\nNome da Organização:', organizacao_experiencia,
      '\nTipo de Contratação:', tipo_contrato_experiencia,
      '\nData de Início:', data_inicio_experiencia,
      '\nData de Término:', data_fim_experiencia,
      '\nDuração da Experiência:', duracao_experiencia)

Título registrado na última experiência profissional: Co-Founder 
Nome da Organização: Nubank 
Tipo de Contratação:  
Data de Início: mai de 2013  
Data de Término:  o momento  
Duração da Experiência:  9 anos 7 meses


## Extraindo as Informações referente à Formação Acadêmica mais Recente

<center><img alt="Exemplo de Formação Acadêmica no LinkedIn" src='https://github.com/octavianosilva/Imagens_projetos/blob/main/exemplo%20de%20forma%C3%A7%C3%A3o%20acad%C3%AAmica.png?raw=true'></center>

Agora vamos extrair as informações da seção Formação Acadêmica, e aqui também temos mais de um formato de estrutura, ou seja, é necessário criar duas funções para pegar os tipos: informações básicas sobre a formação acadêmica e a estrutura em que o usuário descreve as atividades realizadas durante a formação.

Para a Formação Acadêmica, também temos uma página exclusiva, portanto vamos acessá-la:

In [11]:
# Acessando a página de Formação Acadêmica
driver.get('https://www.linkedin.com/in/crisjunqueira/details/education/')

# Lendo a página do início ao fim
inicio = time.time()
posicao_inicial_rolamento = 0
posicao_final_rolamento = 1000

while True:
    driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")
   
    posicao_inicial_rolamento = posicao_final_rolamento
    posicao_final_rolamento += 1000
    
    # Aguardando 3 segundos
    time.sleep(3)
    
    fim = time.time()
    
    # Executar o script por 20 segundos
    if round(fim - inicio) > 20:
        break

# Salvando o código fonte da página em uma variável
src_educacao = driver.page_source

# Utilizando o código fonte para gerar um objeto Beautiful Soup
soup_educacao = BeautifulSoup(src_educacao,'lxml')

In [12]:
# Obtendo o HTML de extração das formações acadêmicas
educacao = soup_educacao.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})

# Visualizando o código HTML
#print(educacao[0])

# Caso queira outras formações mais antigas, basta mudar a posição na variável 'educacao[posição]'

Como temos pessoas que não possuem estudos ou que não informaram nenhuma Formação Acadêmica, vamos precisar criar uma função para contornar essa situação:

In [23]:
# Criando uma função para extrair as informações da Formação Acadêmica mais recente
def extracao_formacao_academica(educacao):
    # Extraindo o nome da Instituição de Ensino
    nome_instituicao_loc = educacao[0].find('span', {'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    nome_instituicao = nome_instituicao_loc.get_text().strip()

    # Separando o Tipo da Formação do Nome do Formação
    tipo_nome_formacao_loc = educacao[0].find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
    tipo_nome_formacao = tipo_nome_formacao_loc.get_text().strip().split(',')

    # Tipo da Formação
    tipo_formacao = tipo_nome_formacao[0].strip()

    # Nome da Formação
    nome_formacao = tipo_nome_formacao[1].strip()

    # Extraindo a duração da Formação
    duracao_formacao_loc = educacao[0].find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
    duracao_formacao = duracao_formacao_loc.get_text().strip()
    
    # Retornando as informações
    return nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao

In [24]:
# Executando a função, se não houver Formação Acadêmica retorna as informações nulas
try:
    nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = extracao_formacao_academica(educacao)
except:
    nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = '','','',''

In [25]:
# Visualizando o resultado da extração
print('Nome da Instituição de Ensino mais recente:', nome_instituicao,
      '\nFormação:', tipo_formacao,
      '\nTítulo da Formação:', nome_formacao,
      '\nDuração da Formação:', duracao_formacao)

Nome da Instituição de Ensino mais recente: Northwestern University - Kellogg School of Management 
Formação: MBA 
Título da Formação: Business 
Duração da Formação: 2007 - 2008


## Extraindo as Informações referentes as Competências

Como essas informações são todas relevantes, iremos armazenar todas as competências em uma lista.

In [13]:
# Acessando a página de Competências
driver.get('https://www.linkedin.com/in/crisjunqueira/details/skills/')

# Lendo a página do início ao fim
inicio = time.time()
posicao_inicial_rolamento = 0
posicao_final_rolamento = 1000

while True:
    driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")
   
    posicao_inicial_rolamento = posicao_final_rolamento
    posicao_final_rolamento += 1000
    
    # Aguardando 3 segundos
    time.sleep(3)
    
    fim = time.time()
    
    # Executar o script por 20 segundos
    if round(fim - inicio) > 20:
        break

# Salvando o código fonte da página em uma variável
src_competencia = driver.page_source

# Utilizando o código fonte para gerar um objeto Beautiful Soup
soup_competencia = BeautifulSoup(src_competencia,'lxml')

In [27]:
# Obtendo o HTML de extração das competências
competencias = soup_competencia.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})

# Visualizando o código HTML
#print(competencias)

In [28]:
# Obtendo uma das competências
competencias_loc = competencias[0].find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
competencia = competencias_loc.get_text().strip()
competencia

'Business Strategy'

In [29]:
# Criando uma lista com as competências do perfil
lista_competencias = []

# Definindo um loop para obter cada competência
for p in range(len(competencias)):
    competencias_loc = competencias[p].find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    competencia_item = competencias_loc.get_text().strip()
    lista_competencias.append(competencia_item)

# Exibindo o resultado da extração de todas as competências
lista_competencias

['Business Strategy',
 'Strategy',
 'Business Planning',
 'Strategic Planning',
 'Financial Modeling',
 'Management Consulting',
 'Cross-functional Team Leadership',
 'Marketing Strategy',
 'Portfolio Management',
 'Management',
 'Segmentation',
 'Product Management',
 'Leadership',
 'Market Research',
 'Business Intelligence',
 'Competitive Analysis',
 'Business Development',
 'Finance',
 'Project Management',
 'Marketing']

## Extraindo as Informações sobre os Idiomas

Também é interessante conhecer se o candidato possui proficiência em outras línguas, portanto vamos aproveitar que a Seção Idiomas também possui uma página exclusiva e extrair essas informações.

In [14]:
# Acessando a página de Idiomas
driver.get('https://www.linkedin.com/in/crisjunqueira/details/languages/')

# Lendo a página do início ao fim
inicio = time.time()
posicao_inicial_rolamento = 0
posicao_final_rolamento = 1000

while True:
    driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")
   
    posicao_inicial_rolamento = posicao_final_rolamento
    posicao_final_rolamento += 1000
    
    # Aguardando 3 segundos
    time.sleep(3)
    
    fim = time.time()
    
    # Executar o script por 20 segundos
    if round(fim - inicio) > 20:
        break

# Salvando o código fonte da página em uma variável
src_idiomas = driver.page_source

# Utilizando o código fonte para gerar um objeto Beautiful Soup
soup_idiomas = BeautifulSoup(src_idiomas,'lxml')

In [15]:
# Obtendo o HTML de extração dos idiomas
idiomas = soup_idiomas.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})

# Visualizando o código HTML
#print(idiomas[0])

In [32]:
# Extraindo o primeiro idioma e acrescentando sua proficiência em '()'
idiomas_loc = idiomas[0].find('span',{'class':'mr1 t-bold'}).find('span',{'aria-hidden':'true'})
idiomas_item = idiomas_loc.get_text().strip()

proficiencia_loc = idiomas[0].find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
proficiencia_item = proficiencia_loc.get_text().strip()

# Unindo as duas informações em uma única String
idioma_proficiencia = '{} ({})'.format(idiomas_item,proficiencia_item)

# Visualizando o resultado
idioma_proficiencia

'English (Nível avançado)'

Agora vamos criar um loop para extrair todas as línguas descritas no perfil, mas como também temos perfis que não possuem uma seção de Idiomas, vamos criar uma função para contornar essa situação:

In [33]:
# Criando uma função para extrair as informações sobre as línguas do candidato
def extraindo_idiomas(idiomas):
    # Criando uma lista com as línguas do perfil
    lista_linguas = []

    # Definindo um loop para obter cada língua
    for p in range(len(idiomas)):
        # Extraindo o nome da língua
        idiomas_loc = idiomas[p].find('span',{'class':'mr1 t-bold'}).find('span',{'aria-hidden':'true'})
        idiomas_item = idiomas_loc.get_text().strip()

        # Extraindo o nível de proficiência
        try:
            proficiencia_loc = idiomas[p].find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
            proficiencia_item = proficiencia_loc.get_text().strip()

            # Unindo as duas informações em uma única String
            idioma_proficiencia = '{} ({})'.format(idiomas_item,proficiencia_item)

        except: # Caso a pessoa não informe o nível de proficiência
            idioma_proficiencia = idiomas_item

        lista_linguas.append(idioma_proficiencia)
        
    # Retornando a lista com os idiomas
    return lista_linguas

In [34]:
# Executando a função para extrair as informações referentes aos idiomas para caso estejam disponíveis
try:
    lista_linguas = extraindo_idiomas(idiomas)
except:
    lista_linguas = ''

In [35]:
# Exibindo o resultado
lista_linguas

['English (Nível avançado)',
 'Portuguese (Fluente ou nativo)',
 'Spanish (Nível básico)']

## Extraindo as Informações referentes aos Prêmios e Honrarias do Candidato

Há pessoas que participam de competições ou recebem premiações como reconhecimento por seus trabalhos que disponibilizam essas conquistas em seus perfis, o que é uma informação valiosa para um recrutador.

Essa Seção de Honrarias também possui uma página exclusiva, portanto vamos acessá-la. Para esse caso iremos utilizar outro perfil já que o atual não possui informações sobre Honrarias.

In [16]:
# Acessando a página de Honrarias
driver.get('https://www.linkedin.com/in/anitta-larissa-machado-a1118818a/details/honors/')

# Lendo a página do início ao fim
inicio = time.time()
posicao_inicial_rolamento = 0
posicao_final_rolamento = 1000

while True:
    driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")
   
    posicao_inicial_rolamento = posicao_final_rolamento
    posicao_final_rolamento += 1000
    
    # Aguardando 3 segundos
    time.sleep(3)
    
    fim = time.time()
    
    # Executar o script por 20 segundos
    if round(fim - inicio) > 20:
        break

# Salvando o código fonte da página em uma variável
src_honras = driver.page_source

# Utilizando o código fonte para gerar um objeto Beautiful Soup
soup_honras = BeautifulSoup(src_honras,'lxml')

In [17]:
# Obtendo o HTML de extração das honrarias
honras = soup_honras.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})

# Visualizando o código HTML
#print(honras[0])

In [38]:
# Extraindo a primeira honraria e acrescentando a data em '()'
honra_loc = honras[0].find('span',{'class':'mr1 t-bold'}).find('span',{'aria-hidden':'true'})
honra_item = honra_loc.get_text().strip()

# Extraindo a data da honraria
try:
    data_loc = honras[0].find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
    data_item = data_loc.get_text().strip()

    # Unindo as duas informações em uma única String
    honra_data = '{} ({})'.format(honra_item,data_item)

except: # Caso a pessoa não informe a data da honraria
    honra_data = honra_item

# Visualizando o resultado
honra_data

'Prêmio ASCAP Latin Music Awards (jan de 2019)'

Como temos pessoas que não possuem essa informação disponível, temos que criar uma função para contornar a situação e também para retornar uma lista com todas as honrarias:

In [39]:
# Criando uma função para extrair as informações sobre as honrarias do candidato
def extraindo_honrarias(honras):
    # Criando uma lista para as honrarias do perfil
    lista_honras = []

    # Definindo um loop para obter cada honraria
    for p in range(len(honras)):
        # Extraindo a primeira honraria e acrescentando a data em '()'
        honra_loc = honras[p].find('span',{'class':'mr1 t-bold'}).find('span',{'aria-hidden':'true'})
        honra_item = honra_loc.get_text().strip()

        # Extraindo a data da honraria
        try:
            data_loc = honras[p].find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
            data_item = data_loc.get_text().strip()

            # Unindo as duas informações em uma única String
            honra_data = '{} ({})'.format(honra_item,data_item)

        except: # Caso a pessoa não informe a data da honraria
            honra_data = honra_item

        lista_honras.append(honra_data)
        
    # Retornando a lista com as honrarias
    return lista_honras

In [40]:
# Executando a função
try:
    lista_honras = extraindo_honrarias(honras)
except: # Caso o perfil não possua honrarias
    lista_honras = ''

In [41]:
# Visualizando o resultado
lista_honras

['Prêmio ASCAP Latin Music Awards (jan de 2019)',
 'Prêmio Multishow de Música Brasileira (jan de 2019)',
 'Troféu Imprensa SBT (jan de 2019)',
 'Jingle Bells Awards (jan de 2018)',
 'Latin American Music Awards (jan de 2018)',
 'Latin Music Italian Awards de Milão/Itália (jan de 2018)',
 'MTV Millennial Awards México (jan de 2018)',
 'Melhores do Ano da Rede Globo (jan de 2018)',
 'Meus Prêmios Nick (jan de 2018)',
 'Prêmio "Melhor Clipe do Ano" do MTV Israel: Desfile Anual (jan de 2018)',
 'Prêmio F5 da Folha de São Paulo (jan de 2018)',
 'Prêmio Faz Diferença (jan de 2018)',
 'Prêmio MTV Europe Music Award (jan de 2018)',
 'Prêmio MTV Millennial Awards Brasil (jan de 2018)',
 'Prêmio TeleHit (jan de 2018)',
 'Prêmio da Revista Mexicana "Eres" (jan de 2018)',
 'iHeartRadio Music Awards (jan de 2018)',
 'Capricho Awards (jan de 2017)',
 'Melhores do Ano FM O Dia (jan de 2017)',
 'Prêmio CONTIGO! Online (jan de 2017)']

## Extraindo a Quantidade de Recomendações o Candidato possui

Também temos aqueles perfis que possuem recomendações de outros profissionais, o que é outra informação valiosa para o recrutador.

Como também temos uma página exclusiva para essa Seção de Recomendações, fica mais fácil extrair essas informações.

Aqui também usaremos outro perfil que possui recomendações, para demonstrar o funcionamento.

In [42]:
# Acessando a página de Recomendações
driver.get('https://www.linkedin.com/in/shilpar/details/recommendations/')

# Lendo a página do início ao fim
inicio = time.time()
posicao_inicial_rolamento = 0
posicao_final_rolamento = 1000

while True:
    driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")
   
    posicao_inicial_rolamento = posicao_final_rolamento
    posicao_final_rolamento += 1000
    
    # Aguardando 3 segundos
    time.sleep(3)
    
    fim = time.time()
    
    # Executar o script por 20 segundos
    if round(fim - inicio) > 20:
        break

# Salvando o código fonte da página em uma variável
src_recomendacoes = driver.page_source

# Utilizando o código fonte para gerar um objeto Beautiful Soup
soup_recomendacoes = BeautifulSoup(src_recomendacoes,'lxml')

In [43]:
# Obtendo o HTML de extração das recomendações
recomendacoes = soup_recomendacoes.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})

# Obtendo a quantidade de Recomendações do Perfil
print(len(recomendacoes))

4


Como nem todos os perfis possuem recomendações precisamos criar uma condição, além disso mesmo que o perfil não possua nenhuma recomendação, ele receberá o valor 1, ou seja, estaria sendo computada uma recomendação de forma errada.

Isso se dá pelo fato de que mesmo sem nenhuma recomendação, o código HTML possui ao menos um elemento 'li', que quando chamandos `len()` retorna o valor 1. Para contornar esse problema iremos aprofundar no código HTML apenas para validar se há uma recomendação ou não.

Resumindo, se não houver recomendações, o código tentará acessar um elemento que não existe, causando erro e saindo do `TRY` para o `EXCEPT`, que então retornará o valor 0 corretamente. Caso haja uma recomendação, o elemento será encontrado e o `TRY` irá prosseguir normalmente. 

In [44]:
# Vamos acessar a primeira recomendação somente para validar
validando_recomendacoes = recomendacoes[0].find('span',{'class':'mr1 hoverable-link-text t-bold'})

# Caso a validação retorne None, recomendações = 0
if validando_recomendacoes == None:

    # Obtendo a quantidade de recomendações do perfil
    qtde_recomendacoes = 0 
    
else: # Caso haja recomendações
    
    # Obtendo a quantidade de recomendações do perfil
    qtde_recomendacoes = len(recomendacoes)

In [45]:
# Exibindo o resultado
qtde_recomendacoes

4

## Definindo uma Pipeline para Extrair as Informações de vários Candidatos

Agora que vimos que é possível extrair as informações para uma única pessoa, vamos definir uma pipeline para acessar cada URL dos candidatos que aplicaram nas vagas.

### Modulos (área com todas as funções utilizadas na pipeline)

In [46]:
# Função para Logar na Plataforma LinkedIn
def logar_linkedin(login, password):
    # Abrindo a página de Login do LinkedIn
    driver.get('https://www.linkedin.com/login/')
    time.sleep(2) # aguardar 2 segundos para a página abrir

    # Inserindo o usuário
    usuario = driver.find_element('id','username') # acessando o campo usuário
    usuario.send_keys(login) # enviando a string com o usuário
    
    # Inserindo a senha
    senha = driver.find_element('id','password') # acessando o campo senha
    senha.send_keys(password) # enviando a string com a senha
    
    # Clicando no botão 'Entrar' para acessar a conta no LinkedIn
    driver.find_element("xpath","//button[@type='submit']").click()

In [47]:
# Função para baixar o Perfil em formato PDF (estilo Currículo)
def baixar_perfil_pdf():
    try:
        try:
            # Clicando no botão '...' ou 'Mais' na seção de "Introdução" do perfil
            driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[3]/button').click()                               
            time.sleep(1) # aguardar 1 segundo

            # Clicando no botão "Salvar como PDF"
            driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[3]/div/div/ul/li[3]/div').click()
        
        except:
            # Clicando no botão '...' ou 'Mais' na seção de "Introdução" do perfil
            driver.find_element('xpath','/html/body/div[4]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[3]/button').click()
            time.sleep(1) # aguardar 1 segundo

            # Clicando no botão "Salvar como PDF"
            driver.find_element('xpath','/html/body/div[4]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[3]/div/div/ul/li[3]/div').click()
            
    except:
        try:
            try:
                # Clicando no botão '...' ou 'Mais' na seção de "Introdução" do perfil
                driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button').click()
                time.sleep(1) # aguardar 1 segundo

                # Clicando no botão "Salvar como PDF"
                driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[3]/div').click()
            
            except:
                # Clicando no botão '...' ou 'Mais' na seção de "Introdução" do perfil
                driver.find_element('xpath','/html/body/div[4]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[3]/button').click()           
                time.sleep(1) # aguardar 1 segundo

                # Clicando no botão "Salvar como PDF"
                driver.find_element('xpath','/html/body/div[4]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[3]/div/div/ul/li[3]/div').click()

        except:
            # Clicando no botão '...' ou 'Mais' na seção de "Introdução" do perfil
            driver.find_element('xpath','/html/body/div[4]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button').click()
            time.sleep(1) # aguardar 1 segundo

            # Clicando no botão "Salvar como PDF"
            driver.find_element('xpath','/html/body/div[4]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[3]/div').click()

In [48]:
# Função para ler a página inicial
def ler_pagina_principal():
    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 2 segundos
        time.sleep(2)

        fim = time.time()

        # Executar o script por 10 segundos
        if round(fim - inicio) > 10:
            break
    
    # Salvando o código fonte da página em uma variável
    src = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup = BeautifulSoup(src,'lxml')
    
    # Retornando o Código Fonte
    return soup

In [49]:
# Função para Extrair as informações contidas na seção de Introdução da página
def extrair_introducao(codigo_fonte_principal):
    # Obtendo o HTML de extração da introdução
    intro = codigo_fonte_principal.find('div',{'class': 'mt2 relative'})
    
    # Localizando e extraindo o nome do candidato
    nome_loc = intro.find('h1')
    nome = nome_loc.get_text().strip()

    # Localizando e extraindo o título profissional do candidato
    titulo_loc = intro.find('div', {'class': 'text-body-medium break-words'})
    titulo = titulo_loc.get_text().strip()

    # Localizando e extraindo o endereço de residência
    try:
        localizacao_loc = intro.find('span', {'class': 'text-body-small inline t-black--light break-words'})
        localizacao = localizacao_loc.get_text().strip()
    except:
        localizacao = ''

    # Localizando e extraindo a organização de trabalho
    trabalho_loc = intro.find('div', {'class': 'inline-show-more-text'})
    if trabalho_loc == None:
        trabalho = ''
    else:
        trabalho = trabalho_loc.get_text().strip()

    # Localizando e extraindo a instituição da formação acadêmica
    estudo_loc = intro.find_all('div', {'class': 'inline-show-more-text'})
    if estudo_loc == None:
        estudo = ''
    else:
        try:
            estudo = estudo_loc[1].get_text().strip()
        except:
            estudo = ''
        
    # Retornando as informações extraídas
    return nome, titulo, localizacao, trabalho, estudo

In [50]:
# Função para extrair a quantidade de Seguidores da página
def extrair_seguidores(codigo_fonte_principal):
    # Obtendo o HTML de extração da quantidade de seguidores
    try:
        networking = codigo_fonte_principal.find('ul', {'class':'pv-top-card--list pv-top-card--list-bullet display-flex mt2'}).find('li',{'class':'text-body-small t-black--light inline-block'})

        # Localizando e extraindo a quantidade de seguidores
        seguidores_loc = networking.find('span', {'class': 't-bold'})
        seguidores = seguidores_loc.get_text().strip()
        
    except:
        networking = codigo_fonte_principal.find('p', {'class':'pvs-header__subtitle text-body-small'})

        # Localizando e extraindo a quantidade de seguidores
        seguidores_loc = networking.find('span', {'aria-hidden': 'true'})
        seguidores = seguidores_loc.get_text().strip()
    
    # Retornando a informação extraída
    return seguidores

In [51]:
# Função para extrair a Biografia da página
def extrair_biografia(codigo_fonte_principal):
    # Obtendo o HTML de extração da biografia do candidato
    sobre_descricao = codigo_fonte_principal.find('div',{'class':'inline-show-more-text inline-show-more-text--is-collapsed'})
    
    # Localizando e extraindo a descrição de perfil na seção Sobre da página
    if sobre_descricao == None:
        sobre = ''
    else:
        sobre_loc = sobre_descricao.find('span', {'aria-hidden': 'true'})
        sobre = sobre_loc.get_text().strip()
        
    # Retornando as informações extraídas
    return sobre

In [52]:
# Função para extrair a experiência profissional TIPO 1
def extrair_info_experiencias(experiencia):
    # Obtendo o título da experiência mais recente
    titulo_experiencia_loc = experiencia[0].find('span',{'class':'mr1 t-bold'}).find('span',{'aria-hidden':'true'})
    titulo_experiencia = titulo_experiencia_loc.get_text().strip()
    
    # Separando o nome da organização e o tipo de contrato
    organizacao_tipo_experiencia_loc = experiencia[0].find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
    organizacao_tipo_experiencia = organizacao_tipo_experiencia_loc.get_text().strip()
    organizacao_tipo_experiencia = organizacao_tipo_experiencia.split('·')
    
    # Nome da organização mais recente
    organizacao_experiencia = organizacao_tipo_experiencia[0]

    # As vezes também temos perfis que não possuem o tipo da contratação especificada no perfil, portanto precisamos de mais um TRY/EXCEPT para contornar esse problema
    try:
        tipo_contrato_experiencia = organizacao_tipo_experiencia[1].strip()
    except:
        tipo_contrato_experiencia = ''
    
    # Separando as datas de início e fim da duração da experiência
    tempo_duracao_experiencia_loc = experiencia[0].find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
    tempo_duracao_experiencia = tempo_duracao_experiencia_loc.get_text().strip().split('·')
    tempo_experiencia = tempo_duracao_experiencia[0].split('-')
    
    # Data de início e fim da experiência
    data_inicio_experiencia = tempo_experiencia[0].strip()
    data_fim_experiencia = tempo_experiencia[1].strip()
    
    # Duração da experiência
    duracao_experiencia = tempo_duracao_experiencia[1].strip()
    
    # Retornando as variáveis com as informações extraídas
    return titulo_experiencia, organizacao_experiencia, tipo_contrato_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia

In [53]:
# Função para extrair a experiência profissional TIPO 2
def extrair_info_experiencias_2(experiencia):
    # Obtendo o nome da organização da experiência mais recente
    organizacao_experiencia_loc = experiencia[0].find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    organizacao_experiencia = organizacao_experiencia_loc.get_text().strip()
    
    # Obtendo o tipo de contrato da experiência (nos casos de pessoas que não tenham essa informação, podem aparecer informações erradas)
    tipo_experiencia_loc = experiencia[0].find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
    tipo_experiencia = tipo_experiencia_loc.get_text().strip().split('·')
    tipo_experiencia = tipo_experiencia[0].strip()
    
    # Obtendo o título mais recente da experiência
    titulo_experiencia_loc = experiencia[0].find_all('a')
    titulo_experiencia_loc = titulo_experiencia_loc[2].find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    titulo_experiencia = titulo_experiencia_loc.get_text().strip()
    
    # Separando as datas de início e fim da duração da experiência
    tempo_duracao_experiencia_loc = experiencia[0].find_all('a')
    tempo_duracao_experiencia_loc = tempo_duracao_experiencia_loc[2].find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
    tempo_duracao_experiencia_loc = tempo_duracao_experiencia_loc.get_text().strip().split('·')
    tempo_experiencia = tempo_duracao_experiencia_loc[0].split('-')
    
    # Datas de início e fim da experiência
    data_inicio_experiencia = tempo_experiencia[0].strip()
    data_fim_experiencia = tempo_experiencia[1].strip()
    
    # Duração da experiência
    duracao_experiencia = tempo_duracao_experiencia_loc[1]
    
    # Retorne as variáveis com as informações extraídas
    return titulo_experiencia, organizacao_experiencia, tipo_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia

In [54]:
# Função para extrair as informações referentes à última experiência profissional
def extrair_experiencia_profissional(perfil_id):
    # Acessando a página referente às experiências profissionais
    experiencia_url = 'https://www.linkedin.com/in/{}/details/experience/'.format(perfil_id)
    driver.get(experiencia_url)

    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 2 segundos
        time.sleep(2)

        fim = time.time()

        # Executar o script por 5 segundos
        if round(fim - inicio) > 5:
            break

    # Salvando o código fonte da página em uma variável
    src_experiencia = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup_experiencia = BeautifulSoup(src_experiencia,'lxml')
    
    # Obtendo o HTML de extração das experiências
    experiencia = soup_experiencia.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})

    # Executando as funções
    try:
        try:
            titulo_experiencia, organizacao_experiencia, tipo_contrato_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia = extrair_info_experiencias(experiencia)
        except:
            titulo_experiencia, organizacao_experiencia, tipo_contrato_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia = extrair_info_experiencias_2(experiencia)
    except:
        titulo_experiencia, organizacao_experiencia, tipo_contrato_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia = ' ',' ',' ',' ',' ',' '
        
    # Retornando as informações extraídas
    return titulo_experiencia, organizacao_experiencia, tipo_contrato_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia

In [55]:
# Função para extrair as informações da Formação Acadêmica mais recente
def extrair_info_formacao_academica(educacao):
    # Extraindo o nome da Instituição de Ensino
    nome_instituicao_loc = educacao[0].find('span', {'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    nome_instituicao = nome_instituicao_loc.get_text().strip()

    # Separando o Tipo da Formação do Nome do Formação
    tipo_nome_formacao_loc = educacao[0].find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
    tipo_nome_formacao = tipo_nome_formacao_loc.get_text().strip().split(',')

    # Tipo da Formação
    tipo_formacao = tipo_nome_formacao[0].strip()

    # Nome da Formação
    nome_formacao = tipo_nome_formacao[1].strip()

    # Extraindo a duração da Formação
    duracao_formacao_loc = educacao[0].find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
    duracao_formacao = duracao_formacao_loc.get_text().strip()
    
    # Retornando as informações
    return nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao

In [56]:
# Função para extrair as informações da Formação Acadêmica mais recente TIPO 2 (onde não tem o nome da formação ou tipo dela)
def extrair_info_formacao_academica_2(educacao):
    # Extraindo o nome da Instituição de Ensino
    nome_instituicao_loc = educacao[0].find('span', {'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    nome_instituicao = nome_instituicao_loc.get_text().strip()

    # Extraindo a duração da Formação (caso esteja disponível)
    try:
        duracao_formacao_loc = educacao[0].find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
        duracao_formacao = duracao_formacao_loc.get_text().strip()
    except:
        duracao_formacao = ''
    
    # Campos não disponíveis `nome_formacao` e `tipo_formacao`
    nome_formacao = ''
    tipo_formacao = ''
    
    # Retornando as informações
    return nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao

In [57]:
# Função para extrair as informações referente à Formação Acadêmica mais recente
def extrair_formacao_academica(perfil_id):
    # Acessando a página referente às formações acadêmicas
    formacao_url = 'https://www.linkedin.com/in/{}/details/education/'.format(perfil_id)
    driver.get(formacao_url)
    
    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 2 segundos
        time.sleep(2)

        fim = time.time()

        # Executar o script por 5 segundos
        if round(fim - inicio) > 5:
            break

    # Salvando o código fonte da página em uma variável
    src_educacao = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup_educacao = BeautifulSoup(src_educacao,'lxml')
    
    # Obtendo o HTML de extração das formações acadêmicas
    educacao = soup_educacao.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})

    # Executando a função, se não houver Formação Acadêmica retorna as informações nulas
    try:
        try:
            nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = extrair_info_formacao_academica(educacao)
        except:
            nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = extrair_info_formacao_academica_2(educacao)
    except:
        nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = '','','',''
        
    # Retornando as informações extraídas
    return nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao

In [58]:
# Função para extrair a lista de Competências do Candidato
def extrair_competencias(perfil_id):
    # Acessando a página de Competências
    competencias_url = 'https://www.linkedin.com/in/{}/details/skills/'.format(perfil_id)
    driver.get(competencias_url)

    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 2 segundos
        time.sleep(2)

        fim = time.time()

        # Executar o script por 10 segundos
        if round(fim - inicio) > 10:
            break

    # Salvando o código fonte da página em uma variável
    src_competencia = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup_competencia = BeautifulSoup(src_competencia,'lxml')
    
    # Obtendo o HTML de extração das competências
    competencias = soup_competencia.find('ul',{'class':'pvs-list'})

    # Criando uma lista com as competências do perfil
    lista_competencias = []

    # Caso a pessoa não tenha nenhuma competência disponível, o código HTML retornará 'None' 
    if competencias == None:
        lista_competencias = ''
    
    else: # Se o código HTML não for 'None', acessar o HTML com as lista das competências
        # Acessando o HTML das listas das competências
        competencias = soup_competencia.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})
        
        # Definindo um loop para extrair cada competência disponível
        for p in range(len(competencias)):
            competencias_loc = competencias[p].find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
            competencia_item = competencias_loc.get_text().strip()
            lista_competencias.append(competencia_item)

    # Retornando a lista das competências
    return lista_competencias

In [59]:
# Função para contornar o problema de páginas que não possuem Idiomas
def extrair_info_idiomas(idiomas):
    # Criando uma lista com as línguas do perfil
    lista_linguas = []

    # Definindo um loop para obter cada língua
    for p in range(len(idiomas)):
        # Extraindo o nome da língua
        idiomas_loc = idiomas[p].find('span',{'class':'mr1 t-bold'}).find('span',{'aria-hidden':'true'})
        idiomas_item = idiomas_loc.get_text().strip()

        # Extraindo o nível de proficiência
        try:
            proficiencia_loc = idiomas[p].find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
            proficiencia_item = proficiencia_loc.get_text().strip()

            # Unindo as duas informações em uma única String
            idioma_proficiencia = '{} ({})'.format(idiomas_item,proficiencia_item)

        except: # Caso a pessoa não informe o nível de proficiência
            idioma_proficiencia = idiomas_item

        lista_linguas.append(idioma_proficiencia)
        
    # Retornando a lista com os idiomas
    return lista_linguas

In [60]:
# Função para extrair as informações referentes aos Idiomas dominados pelo candidato
def extrair_idiomas(perfil_id):
    # Acessando a página de Idiomas
    idiomas_url = 'https://www.linkedin.com/in/{}/details/languages/'.format(perfil_id)
    driver.get(idiomas_url)

    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 2 segundos
        time.sleep(2)

        fim = time.time()

        # Executar o script por 5 segundos
        if round(fim - inicio) > 5:
            break

    # Salvando o código fonte da página em uma variável
    src_idiomas = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup_idiomas = BeautifulSoup(src_idiomas,'lxml')
    
    # Obtendo o HTML de extração dos idiomas
    idiomas = soup_idiomas.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})
    
    # Executando a função para extrair as informações referentes aos idiomas para caso estejam disponíveis
    try:
        lista_linguas = extrair_info_idiomas(idiomas)
    except:
        lista_linguas = ''
        
    # Retornando a lista com os idiomas do candidato
    return lista_linguas

In [61]:
# Função para contornar o problema com as páginas que não possuem honrarias
def extrair_info_honrarias(honras):
    # Criando uma lista para as honrarias do perfil
    lista_honras = []

    # Definindo um loop para obter cada honraria
    for p in range(len(honras)):
        # Extraindo a primeira honraria e acrescentando a data em '()'
        honra_loc = honras[p].find('span',{'class':'mr1 t-bold'}).find('span',{'aria-hidden':'true'})
        honra_item = honra_loc.get_text().strip()

        # Extraindo a data da honraria
        try:
            data_loc = honras[p].find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
            data_item = data_loc.get_text().strip()

            # Unindo as duas informações em uma única String
            honra_data = '{} ({})'.format(honra_item,data_item)

        except: # Caso a pessoa não informe a data da honraria
            honra_data = honra_item

        lista_honras.append(honra_data)
        
    # Retornando a lista com as honrarias
    return lista_honras

In [62]:
# Função para extrair as Honrarias e Premiações
def extrair_honrarias(perfil_id):
    # Acessando a página de Honrarias
    honrarias_url = 'https://www.linkedin.com/in/{}/details/honors/'.format(perfil_id)
    driver.get(honrarias_url)

    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 3 segundos
        time.sleep(3)

        fim = time.time()

        # Executar o script por 5 segundos
        if round(fim - inicio) > 5:
            break

    # Salvando o código fonte da página em uma variável
    src_honras = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup_honras = BeautifulSoup(src_honras,'lxml')
    
    # Obtendo o HTML de extração das honrarias
    honras = soup_honras.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})
    
    # Executando a função
    try:
        lista_honras = extrair_info_honrarias(honras)
    except: # Caso o perfil não possua honrarias
        lista_honras = ''
        
    # Retornando a lista com as honrarias
    return lista_honras

In [63]:
# Função para extrair a quantidade de Recomendações que o candidato possui
def extrair_recomendacoes(perfil_id):
    # Acessando a página de Recomendações
    recomendacoes_url = 'https://www.linkedin.com/in/{}/details/recommendations/'.format(perfil_id)
    driver.get(recomendacoes_url)

    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 3 segundos
        time.sleep(3)

        fim = time.time()

        # Executar o script por 5 segundos
        if round(fim - inicio) > 5:
            break

    # Salvando o código fonte da página em uma variável
    src_recomendacoes = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup_recomendacoes = BeautifulSoup(src_recomendacoes,'lxml')
    
    # Obtendo o HTML de extração das recomendações
    recomendacoes = soup_recomendacoes.find('ul',{'class':'pvs-list'}).find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})

    # Vamos acessar a primeira recomendação somente para validar
    validando_recomendacoes = recomendacoes[0].find('span',{'class':'mr1 hoverable-link-text t-bold'})

    # Caso a validação retorne None, recomendações = 0
    if validando_recomendacoes == None:

        # Obtendo a quantidade de recomendações do perfil
        qtde_recomendacoes = 0 
    
    else: # Caso haja recomendações
    
        # Obtendo a quantidade de recomendações do perfil
        qtde_recomendacoes = len(recomendacoes)
          
    # Retornando a informação extraída
    return qtde_recomendacoes

In [64]:
# Função de Execução da Pipeline
def executar_webscraping(login, password, lista_candidatos):
    
    # Logando no LinkedIn
    logar_linkedin(login, password)
    
    # Estruturando o DataFrame para receber as informações da extração
    df = pd.DataFrame(columns=['Nome Candidato', 'Título do Candidato', 'Localização', 'Organização Atual', 'Estudou em', 'Quantidade de Seguidores',
                               'Biografia', 'Última Experiência', 'Tipo da Experiência', 'Cargo da Experiência', 'Início da Experiência', 'Fim da Experiência',
                               'Duração da Experiência', 'Última Instituição Acadêmica', 'Tipo da Formação', 'Formação', 'Duração da Formação', 'Lista de Competências',
                               'Lista de Idiomas', 'Lista de Honrarias', 'Quantidade de Recomendações', 'Data de Injestão', 'URL do Perfil'])
    
    # Definindo um Loop para acessar cada perfil de candidato disponível
    for perfil in lista_candidatos:
        
        # Acessando a URL do perfil do candidato listado
        perfil_url = perfil
        driver.get(perfil_url)
        time.sleep(2) # aguarde 2 segundos para a página abrir
        
        # Baixando perfil em formato PDF
        #baixar_perfil_pdf()
        
        # Lendo a página principal
        codigo_fonte_principal = ler_pagina_principal()
        
        # Extraindo as informações da seção de Introdução da página
        nome_candidato, titulo_intro, localizacao, trabalho_intro, estudo_intro = extrair_introducao(codigo_fonte_principal)
        
        # Extraindo a quantidade de Seguidores
        qtde_seguidores = extrair_seguidores(codigo_fonte_principal)
        
        # Extraindo a Biografia
        biografia = extrair_biografia(codigo_fonte_principal)
        
        # Obtendo o ID do perfil para acessar as páginas de Experiência, Formação, Idiomas, etc
        perfil_id = perfil_url.split('/')[4]
        
        # Extraindo as informações da seção Experiência Profissional
        titulo_experiencia, organizacao_experiencia, tipo_contrato_experiencia, data_inicio_experiencia, data_fim_experiencia, duracao_experiencia = extrair_experiencia_profissional(perfil_id)
        
        # Extraindo as informações da seção Formação Acadêmica
        nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = extrair_formacao_academica(perfil_id)
        
        # Extraindo a lista das Competências do perfil
        lista_competencias = extrair_competencias(perfil_id)
        
        # Extraindo a lista com os Idiomas falados pelo candidato
        lista_linguas = extrair_idiomas(perfil_id)
        
        # Extraindo a lista com as Honrarias recebidas pelo candidato
        lista_honras = extrair_honrarias(perfil_id)
        
        # Extraindo a quantidade de Recomendações que o candidato possui
        qtde_recomendacoes = extrair_recomendacoes(perfil_id)
        
        # Obtendo a Data em que o código foi executado
        data_ingestao = date.today()
        data_ingestao = data_ingestao.strftime("%d/%m/%Y")
        
        # Gerando um DataFrame com as informações
        dicionario = {'Nome Candidato': [nome_candidato],
                       'Título do Candidato': [titulo_intro],
                       'Localização': [localizacao],
                       'Organização Atual': [trabalho_intro],
                       'Estudou em': [estudo_intro],
                       'Quantidade de Seguidores': [qtde_seguidores],
                       'Biografia': [biografia],
                       'Última Experiência': [organizacao_experiencia],
                       'Tipo da Experiência': [tipo_contrato_experiencia],
                       'Cargo da Experiência': [titulo_experiencia],
                       'Início da Experiência': [data_inicio_experiencia],
                       'Fim da Experiência': [data_fim_experiencia],
                       'Duração da Experiência': [duracao_experiencia],
                       'Última Instituição Acadêmica': [nome_instituicao],
                       'Tipo da Formação': [tipo_formacao],
                       'Formação': [nome_formacao],
                       'Duração da Formação': [duracao_formacao],
                       'Lista de Competências': [lista_competencias],
                       'Lista de Idiomas': [lista_linguas],
                       'Lista de Honrarias': [lista_honras],
                       'Quantidade de Recomendações': [qtde_recomendacoes],
                       'Data de Injestão': [data_ingestao],
                       'URL do Perfil': [perfil]}
        
        index_dicionario = [0]
        
        # Convertendo o Dicionário em um DataFrame
        df_dicionario = pd.DataFrame(dicionario, index=index_dicionario)

        
        # Adicionando as informações extraídas dos perfis no DataFrame
        df = pd.concat([df, df_dicionario], ignore_index=True)
        
    # Retornando o DataFrame com os dados extraídos
    return df

In [65]:
# Informando o Caminho do Web Driver, Login, Senha, Lista das URLs dos candidatos
login = 'SEU LOGIN'
password = 'SUA SENHA'
lista_candidatos = ['https://www.linkedin.com/in/williamhgates/',
                    'https://www.linkedin.com/in/anitta-larissa-machado-a1118818a/',
                    'https://www.linkedin.com/in/crisjunqueira/',
                    'https://www.linkedin.com/in/shilpar/']

In [66]:
# Instanciando o Web Driver
srv = Service('CAMINHO DO WEB DRIVER\chromedriver.exe')
opt = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=srv, options=opt)

In [67]:
# Executando o script
df_dados = executar_webscraping(login, password, lista_candidatos)

In [68]:
# Exibindo o resultado
pd.set_option('max_columns', None)
df_dados

,Nome Candidato,Título do Candidato,Localização,Organização Atual,Estudou em,Quantidade de Seguidores,Biografia,Última Experiência,Tipo da Experiência,Cargo da Experiência,Início da Experiência,Fim da Experiência,Duração da Experiência,Última Instituição Acadêmica,Tipo da Formação,Formação,Duração da Formação,Lista de Competências,Lista de Idiomas,Lista de Honrarias,Quantidade de Recomendações,Data de Injestão,URL do Perfil
0,Bill Gates,"Co-chair, Bill & Melinda Gates Foundation","Seattle, Washington, Estados Unidos",Bill & Melinda Gates Foundation,Harvard University,36.033.134 seguidores,Co-chair of the Bill & Melinda Gates Foundatio...,Bill & Melinda Gates Foundation,,Co-chair,2000,o momento,22 anos 11 meses,Harvard University,,,1973 - 1975,,,,0,16/11/2022,https://www.linkedin.com/in/williamhgates/
1,Anitta Larissa Machado,Embaixadora Global do Nubank,"Rio de Janeiro, Rio de Janeiro, Brasil",Estácio,,235.346 seguidores,"Em 2013, Anitta firmou seu nome no cenário mus...",Estácio,,Criadora do Curso Anitta Prepara,mai de 2022,o momento,7 meses,,,,,"[Cantora, Spanish, Marketing, English, Portugu...","[English, Portuguese, Spanish]",[Prêmio ASCAP Latin Music Awards (jan de 2019)...,0,16/11/2022,https://www.linkedin.com/in/anitta-larissa-mac...
2,Cristina Junqueira,Co-Founder at Nubank,"São Paulo, São Paulo, Brasil",Nubank,Northwestern University - Kellogg School of Ma...,505.612 seguidores,Selected to One Year Accelerated ProgramMajors...,Nubank,,Co-Founder,mai de 2013,o momento,9 anos 7 meses,Northwestern University - Kellogg School of Ma...,MBA,Business,2007 - 2008,"[Business Strategy, Strategy, Business Plannin...","[English (Nível avançado), Portuguese (Fluente...",,0,16/11/2022,https://www.linkedin.com/in/crisjunqueira/
3,Shilpa Ranganathan,"Corporate Vice President, Windows at Microsoft",Seattle e Região,Microsoft,"Birla Institute of Technology and Science, Pilani",2.153 seguidores,"Specialties: Program Management, Project Manag...",Microsoft,14 a 7 m,"Corporate Vice President, Windows",abr de 2022,o momento,8 meses,"Birla Institute of Technology and Science, Pilani",B.E,Electrical & Electronics,1995 - 1999,"[Software Development, Program Management, Sof...",,,4,16/11/2022,https://www.linkedin.com/in/shilpar/


In [69]:
# Salvando os Resultados em uma Planilha Excel
df_dados.to_excel("Resultados Web Scraping.xlsx")

Pelo DataFrame acima é possível ver que a extração foi um sucesso, lembrando que os campos em branco são devido à ausência da informação pelos próprios usuários e a coluna `Tipo da Experiência` precisa passar por um tratamento, uma vez que pela estrutura da página acaba sendo extraída a duração geral da experiência de forma errada, como vemos na quarta linha do DataFrame.

# Conclusão

Com o Script desenvolvido neste projeto concluímos com sucesso nosso objetivo de levar as informações dos candidatos das vagas de emprego para uma única tabela, que contém os dados relevantes para a tomada de decisão do time de Recursos Humanos para selecionar os candidatos mais qualificados.

Como o propósito do projeto era apenas apresentar o funcionamento do Web Scraping para extrair dados valiosos do LinkedIn, não irei explorar as aplicações que poderiam ser feitas com os dados obtidos como resultado do projeto, mas deixo uma provocação: 

***Quais seriam os impactos dessas informações extraídas em um Negócio quando utilizadas em conjunto com modelos de Inteligência Artificial? Direcionamento de Marketing? Recrutamento? Prevenção? Recomendação? Etc.***

Eu sei que provavelmente você já deve ter tido várias ideias de como usar essas informações ao longo desse projeto e agora com essa provocação podem ter surgido ainda mais ideias brilhantes, porém é preciso ter em mente que a Plataforma LinkedIn sabe o quão valiosas essas informações são e por isso vem dificultado cada vez mais a extração via Scraping. Portanto, hoje o script desenvolvido pode funcionar perfeitamente, mas amanhã pode ser que nada mais possa ser aproveitado do processo.

Dito isso, é importante relembrar o que foi dito no início do projeto, a Plataforma vem tomando medidas para detectar o uso de bots ou qualquer processo de extração de dados de seus usuários, mesmo estes dados sendo públicos, então tome cuidado e não cometa abusos na Plataforma com qualquer aplicação que seja feita. Embora o Web Scraping não seja o ideal e nem viável para escalar soluções ou produtizar aplicações, a própria Plataforma do LinkedIn possui uma [API Oficial](https://learn.microsoft.com/en-us/linkedin/), que desde 2015 deixou de ser pública (por conta do valor das informações), que entrega informações além das obtidas com o Web Scraping desenvolvido, sendo necessário criar uma aplicação na Plataforma e também se tornar [Parceiro](https://developer.linkedin.com/content/developer/global/en_us/index/partner-programs/apply), onde irá passar por uma seleção burocrática e, mesmo se aprovado, ainda podem haver limitações ou até mesmo planos de assinatura para obter as informações.

# Referências

O que é o Web Scraping:
* https://canaltech.com.br/seguranca/o-que-e-web-scraping/
* https://pplware.sapo.pt/internet/web-scraping-saiba-o-que-e-e-para-que-serve/

Luta do LinkedIn para proibir o Web Scraping:
* https://olhardigital.com.br/2021/06/14/internet-e-redes-sociais/linkedin-ganhar-nova-chance-para-tentar-impedir-coleta-de-dados-publicos-por-rival/
* https://www.natlawreview.com/article/hiq-labs-v-linkedin
* https://www.csoonline.com/article/3662039/hiq-v-linkedin-court-ruling-will-have-a-material-effect-on-privacy.html
* https://www.adweek.com/media/court-sides-with-linkedin-in-data-scraping-lawsuit-vs-hiq-labs/